In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install  '/kaggle/input/requirements/requirements/torch_1.5/torch-1.5.0cu101-cp37-cp37m-linux_x86_64.whl'
!pip install  '/kaggle/input/requirements/requirements/torch_1.5/torchvision-0.6.0cu101-cp37-cp37m-linux_x86_64.whl'
!pip install  '/kaggle/input/requirements/requirements/torch_1.5/yacs-0.1.7-py3-none-any.whl'
!pip install  '/kaggle/input/requirements/requirements/torch_1.5/fvcore-0.1.1.post200513-py3-none-any.whl'
!pip install  '/kaggle/input/requirements/requirements/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl'
!pip install  '/kaggle/input/requirements/requirements/detectron2/detectron2-0.1.3cu101-cp37-cp37m-linux_x86_64.whl'

In [ ]:
!cp -r ../input/globalwheat/wheat/models/yolov5/* .

In [ ]:
!cp -r /kaggle/input/globalwheat/wheat/imports/transforms.py ./

In [ ]:
!pip install --no-deps '../input/weightedboxesfusion/' > /dev/null

In [ ]:
import sys
sys.path.insert(0, "../input/weightedboxesfusion")

In [ ]:
import torchvision
import torchvision.models as models
import torchvision.datasets as dset
import albumentations as A
import numpy as np
import PIL
import glob
import math  
import torch
import cv2
import random
import glob
import torch.nn.functional as F
import pandas as pd
import os
import io
import gc
import csv
import time
import torch
import detectron2
import itertools
import transforms as T
import argparse
import shutil as sh

from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from tqdm.auto import tqdm
from utils.datasets import *
from ensemble_boxes import *
from models.experimental import *
from utils.datasets import *
from utils.utils import *
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode
from detectron2.modeling import build_model
from detectron2.utils.logger import setup_logger
from detectron2.data import DatasetMapper
setup_logger()

%matplotlib inline

In [ ]:
TEST_BATCH_SIZE = 1
DEST_IMG_SIZE = 512
SRC_IMG_SIZE = 1024
NUM_CLASSES = 2

RESNET_PATH1f = "/kaggle/input/faster-rcnn/faster/resnet_rcnn/resnet_rcnn_1f.pt"
RESNET_PATH2f = "/kaggle/input/globalwheat/wheat/models/resnet_rcnn/resnet_rcnn_2f.pt"
RESNET_PATH3f = "/kaggle/input/globalwheat/wheat/models/resnet_rcnn/resnet_rcnn_3f.pt"
RESNET_PATH4f = "/kaggle/input/globalwheat/wheat/models/resnet_rcnn/resnet_rcnn_4f.pt"
RESNET_PATH5f = "/kaggle/input/globalwheat/wheat/models/resnet_rcnn/resnet_rcnn_5f.pt"

VGG_PATH1f = "/kaggle/input/globalwheat/wheat/models/vgg_rcnn/vgg_rcnn_1f.pt"
VGG_PATH2f = "/kaggle/input/globalwheat/wheat/models/vgg_rcnn/vgg_rcnn_2f.pt"
VGG_PATH3f = "/kaggle/input/globalwheat/wheat/models/vgg_rcnn/vgg_rcnn_3f.pt"
VGG_PATH4f = "/kaggle/input/globalwheat/wheat/models/vgg_rcnn/vgg_rcnn_4f.pt"
VGG_PATH5f = "/kaggle/input/globalwheat/wheat/models/vgg_rcnn/vgg_rcnn_5f.pt"

DETECTRON_CFG_PATH_1f = "/kaggle/input/globalwheat/wheat/models/detectrons/detectron2_1f"
DETECTRON_CFG_PATH_2f = "/kaggle/input/globalwheat/wheat/models/detectrons/detectron2_2f"
DETECTRON_CFG_PATH_3f = "/kaggle/input/globalwheat/wheat/models/detectrons/detectron2_3f"
DETECTRON_CFG_PATH_4f = "/kaggle/input/globalwheat/wheat/models/detectrons/detectron2_4f"
DETECTRON_CFG_PATH_5f = "/kaggle/input/globalwheat/wheat/models/detectrons/detectron2_5f"

YOLO_PATH = "../input/globalwheat/wheat/models/yolov5/weights/best_yolov5x_wheat.pt"

F_MEASURES_PATH_FR = "/kaggle/input/f-measures/f_measures_fr.txt"
F_MEASURES_PATH_D2 = "/kaggle/input/f-measures/f_measures_d2.txt"

WHEAT_PATH = "/kaggle/input/global-wheat-detection"
DARKNET_DS_PATH = "/kaggle/input/globalwheat/wheat/yolov5x-wheat-dataset"

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
class WheatDataset(Dataset):
    def __init__(self, root_path, dest_type, transform=None, process="train", divide=False, k=5, k_pos=0, csv=None, csv_coco_format=True, preprocessing=False, return_tensor=True):
        # root_path is the root path of the wheat dataset.
        # dest_type is the requested type for images in the dataset ('bgr', 'gray', 'rgb').
        # transform is the augmentations to pass images through.
        # divide determines whether to divide the dataset into train/validation or not.
        # Wיhen process = "train"/"val" and k, k_pos --> if divide is TRUE --> determine how to divide the paths
        # to train/validation path lists, by k-fold cross validation. k_pos is the position in the list of paths.
        # to extract images from, to create the validation paths. There are k different k_pos within the list of paths.
        # process = "test" causes get_item() method to return only the current image and its id.
        # csv is a path to a csv file, written in the form (different from Kaggle's format): 'image_id', 'x', 'y', 'w', 'h'.
        # If csv is None then the WheatDataset reads Kaggle's csv according to the root_path.
        # preprocessing togather with a "train" process determines whether it is necessary to return image_id from get_item() method or not.
        # csv_coco_format and the given csv argument determine together if a format conversion is needed for using the data.
        # return_tensor determines if the returning arrays of the image and its targets should be a tensor or not 
        
        assert root_path is not dest_type, "Paths need to be unique!"
        assert dest_type in ['rgb', 'bgr', 'gray'], "Invalid types!"
        assert process in ['train', 'val', 'test'], "Invalid process!"
        assert k_pos < k, "Incompatible ratio between k and k_pos!"

        super(WheatDataset, self).__init__()
        self.transform = transform
        self.process = process
        self.dest_type = dest_type
        self.preprocessing = preprocessing
        self.is_pascal = not csv_coco_format
        self.return_tensor = return_tensor

        self.csv = csv
        if self.process != "test":
            self.paths = glob.glob(root_path + "/train/*.jpg")

            if csv is None:
                self.csv = WheatDataset.adjust_csv(root_path + '/train.csv')
            if divide:
                if self.process == "train":
                    before_pos = self.paths[: len(self.paths) * k_pos // k]
                    after_pos = self.paths[len(self.paths) * (k_pos + 1) // k : len(self.paths)]
                    self.paths = before_pos + after_pos
                elif self.process == "val":
                    self.paths = self.paths[len(self.paths) * k_pos // k : len(self.paths) * (k_pos + 1) // k]
        elif self.process == "test":
            self.paths = glob.glob(root_path + "/test/*.jpg")

        if not self.preprocessing:      
            random.shuffle(self.paths) # Shuffle the paths

    
    @staticmethod
    def adjust_csv(path):
        # Given a path for the wheat csv file,
        # Returns the deserved csv to extract data from

        # Reads csv file from path
        new_csv = pd.read_csv(path)

        # Gets an array of arrays of all bboxes in the form [[x], [y], [w], [h]]
        bboxes = np.stack(new_csv['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep = ',')))

        # Saves the values in the respective column
        for i, col in enumerate(['x', 'y', 'w', 'h']):
            new_csv[col] = bboxes[:, i]

        new_csv.drop(columns=['bbox'], inplace=True)
        new_csv.drop(columns=['source'], inplace=True)
        new_csv.drop(columns=['width'], inplace=True)
        new_csv.drop(columns=['height'], inplace=True)

        return new_csv


    def get_compatible_targets(self, img_id, idx):
        # Gets compatible targets by img_id as a key

        if self.process == "train" or self.process == "val":
            boxes, labels, area, iscrowd = list(), list(), 0, list()
            temp_csv = self.csv.loc[self.csv['image_id'] == img_id]
            boxes = temp_csv[['x', 'y', 'w', 'h']].values

            if len(boxes) > 0 and not self.is_pascal:
                # Converting from Coco to Pascal_voc format
                boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
                boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
                area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.ones(boxes.shape[0], dtype=torch.int64) # One class (Wheat)
            iscrowd = torch.zeros(boxes.shape[0], dtype=torch.int64) # suppose all instances are not crowd

            target = dict()
            target["boxes"] = boxes
            target["labels"] = labels
            target["image_id"] = torch.tensor([idx])
            target["area"] = area
            target["iscrowd"] = iscrowd

            return target


    def __getitem__(self, idx):
        p = self.paths[idx]
        x = cv2.imread(p)
        if self.dest_type is 'rgb':
            x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
        elif self.dest_type is 'gray':
            x = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
        
        img_str_id = p[p.rfind('/') + 1 : p.rfind('.')]

        if self.process == "train" or self.process == "val":
            y = self.get_compatible_targets(img_str_id, idx)
            
            if self.transform is not None:
                sample = self.transform(**{
                    'image': x,
                    'bboxes': y['boxes'],
                    'labels': y['labels'],
                    'area': y['area'],
                    'crowd': y['iscrowd'],
                    'id': y['image_id']
                })
                
                x = sample['image']
                y['boxes'] = torch.as_tensor(sample['bboxes'], dtype=torch.float32)
                y['area'] = torch.as_tensor(sample['area'], dtype=torch.float32)
                y['iscrowd'] = torch.as_tensor(sample['crowd'], dtype=torch.int64)
                y['labels'] = torch.as_tensor(sample['labels'], dtype=torch.int64)
                y['image_id'] = torch.as_tensor(sample['id'], dtype=torch.int64)

            if self.return_tensor:
                x, y = T.ToTensor()(x, y)

            if self.preprocessing:
                return x, y, img_str_id
            return x, y
        elif self.process == "test":
            if self.transform is not None:
                sample = self.transform(**{
                    'image': x
                })
                x = sample['image']

            if self.return_tensor:
                to_tensor = A.Compose([ToTensorV2(p=1.0, always_apply=True)])(**{ 'image': x })            
                x = sample['image']

            return x, img_str_id # No label


    def __len__(self):
        return len(self.paths)

In [ ]:
# Wheat validation dataloader creation
wheat_testset = WheatDataset(WHEAT_PATH, dest_type='rgb', transform=None, process="test", return_tensor=False)

# Wheat validation dataloader creation
wheat_testloader = DataLoader(wheat_testset, batch_size=TEST_BATCH_SIZE, shuffle=False, num_workers=8, collate_fn=collate_fn)

In [ ]:
resnet_rcnn_1f = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

resnet_rcnn_1f.roi_heads.box_predictor.cls_score.out_features = NUM_CLASSES
resnet_rcnn_1f.roi_heads.box_predictor.bbox_pred.out_features = NUM_CLASSES * 4

resnet_rcnn_1f.to(device)

resnet_rcnn_checkpoint = torch.load(RESNET_PATH1f, map_location=device)
resnet_rcnn_1f.load_state_dict(resnet_rcnn_checkpoint['model'])

In [ ]:
resnet_rcnn_2f = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

resnet_rcnn_2f.roi_heads.box_predictor.cls_score.out_features = NUM_CLASSES
resnet_rcnn_2f.roi_heads.box_predictor.bbox_pred.out_features = NUM_CLASSES * 4

resnet_rcnn_2f.to(device)

resnet_rcnn_checkpoint = torch.load(RESNET_PATH2f, map_location=device)
resnet_rcnn_2f.load_state_dict(resnet_rcnn_checkpoint['model'])

In [ ]:
resnet_rcnn_3f = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

resnet_rcnn_3f.roi_heads.box_predictor.cls_score.out_features = NUM_CLASSES
resnet_rcnn_3f.roi_heads.box_predictor.bbox_pred.out_features = NUM_CLASSES * 4

resnet_rcnn_3f.to(device)

resnet_rcnn_checkpoint = torch.load(RESNET_PATH3f, map_location=device)
resnet_rcnn_3f.load_state_dict(resnet_rcnn_checkpoint['model'])

In [ ]:
resnet_rcnn_4f = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

resnet_rcnn_4f.roi_heads.box_predictor.cls_score.out_features = NUM_CLASSES
resnet_rcnn_4f.roi_heads.box_predictor.bbox_pred.out_features = NUM_CLASSES * 4

resnet_rcnn_4f.to(device)

resnet_rcnn_checkpoint = torch.load(RESNET_PATH4f, map_location=device)
resnet_rcnn_4f.load_state_dict(resnet_rcnn_checkpoint['model'])

In [ ]:
resnet_rcnn_5f = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

resnet_rcnn_5f.roi_heads.box_predictor.cls_score.out_features = NUM_CLASSES
resnet_rcnn_5f.roi_heads.box_predictor.bbox_pred.out_features = NUM_CLASSES * 4

resnet_rcnn_5f.to(device)

resnet_rcnn_checkpoint = torch.load(RESNET_PATH5f, map_location=device)
resnet_rcnn_5f.load_state_dict(resnet_rcnn_checkpoint['model'])

In [ ]:
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512)), aspect_ratios=((0.5, 1.0, 2.0)))

class BoxHead(torch.nn.Module):
    def __init__(self, model):
        super(BoxHead, self).__init__()
        self.classifier = torch.nn.Sequential(*list(model.classifier._modules.values())[:-1])

    def forward(self, x):
        x = x.flatten(start_dim=1)
        x = self.classifier(x)
        return x

In [ ]:
vgg = models.vgg16(pretrained=False)

vgg_backbone = vgg.features[:-1]
vgg_backbone.out_channels = 512

vgg_box_head = BoxHead(vgg)

vgg_rcnn_1f = torchvision.models.detection.faster_rcnn.FasterRCNN(
    vgg_backbone,
    rpn_anchor_generator = anchor_generator,
    box_roi_pool = roi_pooler,
    box_head = vgg_box_head,
    box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(4096, num_classes=NUM_CLASSES))

# Changing out_features of model according to our number of classes
vgg_rcnn_1f.roi_heads.box_predictor.cls_score.out_features = NUM_CLASSES
vgg_rcnn_1f.roi_heads.box_predictor.bbox_pred.out_features = NUM_CLASSES * 4

vgg_rcnn_1f.to(device)

vgg_rcnn_checkpoint = torch.load(VGG_PATH1f, map_location=(device))
vgg_rcnn_1f.load_state_dict(vgg_rcnn_checkpoint['model'])

In [ ]:
vgg = models.vgg16(pretrained=False)

vgg_backbone = vgg.features[:-1]
vgg_backbone.out_channels = 512

vgg_box_head = BoxHead(vgg)

vgg_rcnn_2f = torchvision.models.detection.faster_rcnn.FasterRCNN(
    vgg_backbone,
    rpn_anchor_generator = anchor_generator,
    box_roi_pool = roi_pooler,
    box_head = vgg_box_head,
    box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(4096, num_classes=NUM_CLASSES))

# Changing out_features of model according to our number of classes
vgg_rcnn_2f.roi_heads.box_predictor.cls_score.out_features = NUM_CLASSES
vgg_rcnn_2f.roi_heads.box_predictor.bbox_pred.out_features = NUM_CLASSES * 4

vgg_rcnn_2f.to(device)

vgg_rcnn_checkpoint = torch.load(VGG_PATH2f, map_location=(device))
vgg_rcnn_2f.load_state_dict(vgg_rcnn_checkpoint['model'])

In [ ]:
vgg = models.vgg16(pretrained=False)

vgg_backbone = vgg.features[:-1]
vgg_backbone.out_channels = 512

vgg_box_head = BoxHead(vgg)

vgg_rcnn_3f = torchvision.models.detection.faster_rcnn.FasterRCNN(
    vgg_backbone,
    rpn_anchor_generator = anchor_generator,
    box_roi_pool = roi_pooler,
    box_head = vgg_box_head,
    box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(4096, num_classes=NUM_CLASSES))

# Changing out_features of model according to our number of classes
vgg_rcnn_3f.roi_heads.box_predictor.cls_score.out_features = NUM_CLASSES
vgg_rcnn_3f.roi_heads.box_predictor.bbox_pred.out_features = NUM_CLASSES * 4

vgg_rcnn_3f.to(device)

vgg_rcnn_checkpoint = torch.load(VGG_PATH3f, map_location=device)
vgg_rcnn_3f.load_state_dict(vgg_rcnn_checkpoint['model'])

In [ ]:
vgg = models.vgg16(pretrained=False)

vgg_backbone = vgg.features[:-1]
vgg_backbone.out_channels = 512

vgg_box_head = BoxHead(vgg)

vgg_rcnn_4f = torchvision.models.detection.faster_rcnn.FasterRCNN(
    vgg_backbone,
    rpn_anchor_generator = anchor_generator,
    box_roi_pool = roi_pooler,
    box_head = vgg_box_head,
    box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(4096, num_classes=NUM_CLASSES))

# Changing out_features of model according to our number of classes
vgg_rcnn_4f.roi_heads.box_predictor.cls_score.out_features = NUM_CLASSES
vgg_rcnn_4f.roi_heads.box_predictor.bbox_pred.out_features = NUM_CLASSES * 4

vgg_rcnn_4f.to(device)

vgg_rcnn_checkpoint = torch.load(VGG_PATH4f, map_location=device)
vgg_rcnn_4f.load_state_dict(vgg_rcnn_checkpoint['model'])

In [ ]:
vgg = models.vgg16(pretrained=False)

vgg_backbone = vgg.features[:-1]
vgg_backbone.out_channels = 512

vgg_box_head = BoxHead(vgg)

vgg_rcnn_5f = torchvision.models.detection.faster_rcnn.FasterRCNN(
    vgg_backbone,
    rpn_anchor_generator = anchor_generator,
    box_roi_pool = roi_pooler,
    box_head = vgg_box_head,
    box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(4096, num_classes=NUM_CLASSES))

# Changing out_features of model according to our number of classes
vgg_rcnn_5f.roi_heads.box_predictor.cls_score.out_features = NUM_CLASSES
vgg_rcnn_5f.roi_heads.box_predictor.bbox_pred.out_features = NUM_CLASSES * 4

vgg_rcnn_5f.to(device)

vgg_rcnn_checkpoint = torch.load(VGG_PATH5f, map_location=device)
vgg_rcnn_5f.load_state_dict(vgg_rcnn_checkpoint['model'])

In [ ]:
rcnns = [resnet_rcnn_1f, resnet_rcnn_2f, resnet_rcnn_3f, resnet_rcnn_4f, resnet_rcnn_5f,
         vgg_rcnn_1f, vgg_rcnn_2f, vgg_rcnn_3f, vgg_rcnn_4f, vgg_rcnn_5f]

In [ ]:
def create_wheat_dicts_for_detectron(dataset_path, k_pos, k=5, dest_type="rgb", transforms=None, process="train"):
    
    ds = WheatDataset(dataset_path, dest_type, transform=transforms, divide=True, preprocessing=True, k=k, k_pos=k_pos, process=process)
    dl = DataLoader(ds, batch_size=TEST_BATCH_SIZE, shuffle=False, num_workers=8, collate_fn=collate_fn)

    dataset_dicts = list()
    for images, targets, image_ids in tqdm(dl): # batch size = 1
        record = dict()
        record["file_name"] = f'{dataset_path}/{process}/{image_ids[0]}.jpg'
        record["image_id"] = targets[0]['image_id']
        record["height"] = images[0].shape[1]
        record["width"] = images[0].shape[2]

        annotations = list()
        for box in targets[0]['boxes']:
            xmin = box[0].item()
            ymin = box[1].item()
            xmax = box[2].item()
            ymax = box[3].item()

            poly = [
                (xmin, ymin), (xmax, ymin), 
                (xmax, ymax), (xmin, ymax)
            ]
            poly = list(itertools.chain.from_iterable(poly))

            anno = {
              "bbox": [xmin, ymin, xmax, ymax],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [poly],
              "category_id": 0, # Wheat
              "iscrowd": 0
            }
            annotations.append(anno)

        record["annotations"] = annotations
        dataset_dicts.append(record)
        
    return dataset_dicts

In [ ]:
for p in tqdm(["train", "val"]):
    DatasetCatalog.register("wheat1f_" + p, lambda p=p: create_wheat_dicts_for_detectron(WHEAT_PATH, k_pos=0, k=5, process=p))
    MetadataCatalog.get("wheat1f_" + p).set(thing_classes=['Wheat'])

statement_metadata = MetadataCatalog.get("wheat1f_train")

In [ ]:
for p in tqdm(["train", "val"]):
    DatasetCatalog.register("wheat2f_" + p, lambda p=p: create_wheat_dicts_for_detectron(WHEAT_PATH, k=5, k_pos=1, process=p))
    MetadataCatalog.get("wheat2f_" + p).set(thing_classes=['Wheat'])

statement_metadata = MetadataCatalog.get("wheat2f_train")

In [ ]:
for p in tqdm(["train", "val"]):
    DatasetCatalog.register("wheat3f_" + p, lambda p=p: create_wheat_dicts_for_detectron(WHEAT_PATH, k=5, k_pos=2, process=p))
    MetadataCatalog.get("wheat3f_" + p).set(thing_classes=['Wheat'])

statement_metadata = MetadataCatalog.get("wheat3f_train")

In [ ]:
for p in tqdm(["train", "val"]):
    DatasetCatalog.register("wheat4f_" + p, lambda p=p: create_wheat_dicts_for_detectron(WHEAT_PATH, k=5, k_pos=3, process=p))
    MetadataCatalog.get("wheat4f_" + p).set(thing_classes=['Wheat'])

statement_metadata = MetadataCatalog.get("wheat4f_train")

In [ ]:
for p in tqdm(["train", "val"]):
    DatasetCatalog.register("wheat5f_" + p, lambda p=p: create_wheat_dicts_for_detectron(WHEAT_PATH, k=5, k_pos=4, process=p))
    MetadataCatalog.get("wheat5f_" + p).set(thing_classes=['Wheat'])

statement_metadata = MetadataCatalog.get("wheat5f_train")

In [ ]:
cfg_1f = get_cfg()

# Gets the pretrained RetinaNet from the given path
cfg_1f.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml"))
cfg_1f.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_101_FPN_3x.yaml")

# Sets number of classes to 1 (Wheat)
cfg_1f.MODEL.RETINANET.NUM_CLASSES = 1

# Allows images without bounding boxes
cfg_1f.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False

# Sets the output directory of the current Detectron2 model
cfg_1f.OUTPUT_DIR = DETECTRON_CFG_PATH_1f

# Loading Detectron2 model
cfg_1f.MODEL.WEIGHTS = os.path.join(cfg_1f.OUTPUT_DIR, "model_final.pth")

# Detectron2 validation dataloader
val_loader_1f = build_detection_test_loader(cfg_1f, "wheat1f_val")

predictor_1f = DefaultPredictor(cfg_1f)

In [ ]:
cfg_2f = get_cfg()

# Gets the pretrained RetinaNet from the given path
cfg_2f.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml"))
cfg_2f.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_101_FPN_3x.yaml")

# Sets number of classes to 1 (Wheat)
cfg_2f.MODEL.RETINANET.NUM_CLASSES = 1

# Allows images without bounding boxes
cfg_2f.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False

# Sets the output directory of the current Detectron2 model
cfg_2f.OUTPUT_DIR = DETECTRON_CFG_PATH_2f

# Loading Detectron2 model
cfg_2f.MODEL.WEIGHTS = os.path.join(cfg_2f.OUTPUT_DIR, "model_final.pth")

# Detectron2 validation dataloader
val_loader_2f = build_detection_test_loader(cfg_2f, "wheat2f_val")

predictor_2f = DefaultPredictor(cfg_2f)

In [ ]:
cfg_3f = get_cfg()

# Gets the pretrained RetinaNet from the given path
cfg_3f.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml"))
cfg_3f.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_101_FPN_3x.yaml")

# Sets number of classes to 1 (Wheat)
cfg_3f.MODEL.RETINANET.NUM_CLASSES = 1

# Allows images without bounding boxes
cfg_3f.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False

# Sets the output directory of the current Detectron2 model
cfg_3f.OUTPUT_DIR = DETECTRON_CFG_PATH_3f

# Loading Detectron2 model
cfg_3f.MODEL.WEIGHTS = os.path.join(cfg_3f.OUTPUT_DIR, "model_final.pth")

# Detectron2 validation dataloader
val_loader_3f = build_detection_test_loader(cfg_3f, "wheat3f_val")

predictor_3f = DefaultPredictor(cfg_3f)

In [ ]:
cfg_4f = get_cfg()

# Gets the pretrained RetinaNet from the given path
cfg_4f.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml"))
cfg_4f.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_101_FPN_3x.yaml")

# Sets number of classes to 1 (Wheat)
cfg_4f.MODEL.RETINANET.NUM_CLASSES = 1

# Allows images without bounding boxes
cfg_4f.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False

# Sets the output directory of the current Detectron2 model
cfg_4f.OUTPUT_DIR = DETECTRON_CFG_PATH_4f

# Loading Detectron2 model
cfg_4f.MODEL.WEIGHTS = os.path.join(cfg_4f.OUTPUT_DIR, "model_final.pth")

# Detectron2 validation dataloader
val_loader_4f = build_detection_test_loader(cfg_4f, "wheat4f_val")

predictor_4f = DefaultPredictor(cfg_4f)

In [ ]:
cfg_5f = get_cfg()

# Gets the pretrained RetinaNet from the given path
cfg_5f.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml"))
cfg_5f.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_101_FPN_3x.yaml")

# Sets number of classes to 1 (Wheat)
cfg_5f.MODEL.RETINANET.NUM_CLASSES = 1

# Allows images without bounding boxes
cfg_5f.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False

# Sets the output directory of the current Detectron2 model
cfg_5f.OUTPUT_DIR = DETECTRON_CFG_PATH_5f

# Loading Detectron2 model
cfg_5f.MODEL.WEIGHTS = os.path.join(cfg_5f.OUTPUT_DIR, "model_final.pth")

# Detectron2 validation dataloader
val_loader_5f = build_detection_test_loader(cfg_5f, "wheat5f_val")

predictor_5f = DefaultPredictor(cfg_5f)

In [ ]:
detectrons = [predictor_1f, predictor_2f, predictor_3f, predictor_4f, predictor_5f]

In [ ]:
gc.collect()
del vgg_rcnn_checkpoint
del resnet_rcnn_checkpoint

In [ ]:
def run_wbf(boxes, scores, image_size=1023, iou_thr=0.5, skip_box_thr=0.7, weights=None):
    #boxes = [prediction[image_index]['boxes'].data.cpu().numpy()/(image_size-1) for prediction in predictions]
    #scores = [prediction[image_index]['scores'].data.cpu().numpy() for prediction in predictions]
    labels = [np.zeros(score.shape[0]) for score in scores]
    boxes = [box/(image_size) for box in boxes]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    #boxes, scores, labels = nms(boxes, scores, labels, weights=[1,1,1,1,1], iou_thr=0.5)
    boxes = boxes*(image_size)
    return boxes, scores, labels

In [ ]:
def TTAImage(image, index):
    image1 = image.copy()
    if index==0: 
        rotated_image = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image
    elif index==1:
        rotated_image2 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image2 = cv2.rotate(rotated_image2, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image2
    elif index==2:
        rotated_image3 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image3
    elif index == 3:
        return image1
    
    
def rotBoxes90(boxes, im_w, im_h):
    ret_boxes =[]
    for box in boxes:
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = x1-im_w//2, im_h//2 - y1, x2-im_w//2, im_h//2 - y2
        x1, y1, x2, y2 = y1, -x1, y2, -x2
        x1, y1, x2, y2 = int(x1+im_w//2), int(im_h//2 - y1), int(x2+im_w//2), int(im_h//2 - y2)
        x1a, y1a, x2a, y2a = min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)
        ret_boxes.append([x1a, y1a, x2a, y2a])
    return np.array(ret_boxes)

In [ ]:
def detect1Image_aug(im0, imgsz, model, device, conf_thres, iou_thres):
    img = letterbox(im0, new_shape=imgsz)[0]
    # Convert
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)


    img = torch.from_numpy(img).to(device)
    img =  img.float()  # uint8 to fp16/32
    img /= 255.0   
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    pred = model(img, augment=True)[0]

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres)

    boxes = []
    scores = []
    for i, det in enumerate(pred):  # detections per image
        # save_path = 'draw/' + image_id + '.jpg'
        if det is not None and len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

            # Write results
            for *xyxy, conf, cls in det:
                boxes.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3])])
                scores.append(conf)

    return np.array(boxes), np.array(scores) 

In [ ]:
# Loading YOLOv5 model, trained on the the original data and additionally on the pseudo test labels
yolov5 = torch.load("/kaggle/input/globalwheat/wheat/models/yolov5/weights/best_yolov5x_wheat.pt", map_location=device)['model'].float()
yolov5.to(device).eval();

gc.collect()

In [ ]:
def draw_bboxes(img, boxes, text=None, text_size=None, thickness=3):
    # Draws all the given bounding boxes (boxes) within the given image (img), using a thickness value,
    # a text list corresponding with each bounding box, and its size
    
    new_img = img.copy()
    for b, t in zip(boxes, text):
        start_point = (int(b[0].item()), int(b[1].item()))
        end_point = (int(b[2].item()), int(b[3].item()))
        color = (0, 255, 255) 
        new_img = cv2.rectangle(new_img, start_point, end_point, color, thickness)
        if text is not None and text_size is not None:
            cv2.putText(new_img, t, (int(b[0].item()), int(b[1].item()) - 2), cv2.FONT_HERSHEY_SIMPLEX, text_size, (0, 255, 255), 4)
    
    return new_img

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

In [ ]:
def test_ensemble(faster_rcnns, detectrons2, yolov5, testloader, faster_rcnn_weights_path, detectron2_weights_path, score_thresh=[0.35, 0.35, 0.35, 0.35], iou_thresh=[0.4, 0.4, 0.4, 0.4]):
    # Given a detection ensemble of models (faster_rcnns, detectron2 models and a yolo), a test dataloader, a path to texts file which include f-measures of each model,
    # and iou/score threshold lists, for each model type: typei --> thresh(i). The thresholds of the whole ensemble is in the last indexes of these lists.
    # Returns a dictionary including the results of the given ensemble according to the submission format of the wheat competition

    results, fr_f_measures, d2_f_measures = list(), list(), list()

    # Reads each f-measure corresponding with each model (five versions (folds) of resnet, 
    # vgg Faster RCNNs or detectrons version 2), by the given path of a text file.
    # These F-Measures are the weights of the models in the ensemble
    with open(faster_rcnn_weights_path, 'r') as eval_file:
        line = eval_file.readline()
        while line:
            fr_f_measures.append(float(line))
            line = eval_file.readline()
    with open(detectron2_weights_path, 'r') as eval_file:
        line = eval_file.readline()
        while line:
            d2_f_measures.append(float(line))
            line = eval_file.readline()
    
    # Collects predictions of each model
    for images, image_ids in tqdm(testloader): # batch size = 1
        # img --> for the predictions of YOLOv5 and Detectron2 models
        img = images[0]
        
        # resized_img --> for the predictions of Faster RCNN models
        # Resizing and normalizing the image to match faster rcnn test predictions (trained on image size of 512 X 512 and range (0-1))
        resized_img = cv2.resize(img, (DEST_IMG_SIZE, DEST_IMG_SIZE), cv2.INTER_LINEAR) / DEST_IMG_SIZE # Resizing image to original size
        
        all_fr_boxes, all_fr_scores, all_fr_labels = list(), list(), list()
        all_d2_boxes, all_d2_scores, all_d2_labels = list(), list(), list()
        all_y_boxes, all_y_scores, all_y_labels = list(), list(), list()

        for fr in faster_rcnns:
            fr.eval()

            tta_fr_boxes, tta_fr_scores, tta_fr_labels = list(), list(), list()

            for i in range(4):
                resized_img_ = torch.from_numpy(TTAImage(resized_img, i)).permute(2, 0, 1).float().to(device) # (3, 512, 512), tensor, 0-1
                
                # Current Faster RCNN prediction using TTA
                fr_prediction = fr([resized_img_])[0] # batch size = 1
                fr_boxes = fr_prediction['boxes'].data.cpu().numpy()
                fr_scores = fr_prediction['scores'].data.cpu().numpy().tolist()

                for _ in range(3-i): # Reordering the boxes
                    fr_boxes = rotBoxes90(fr_boxes, resized_img_.shape[1], resized_img_.shape[1])

                tta_fr_boxes.append((fr_boxes / resized_img_.shape[1]).tolist())
                tta_fr_scores.append(fr_scores)
                tta_fr_labels.append([1] * len(fr_boxes))
                        
            tta_fr_boxes, tta_fr_scores, tta_fr_labels = weighted_boxes_fusion(tta_fr_boxes, tta_fr_scores, tta_fr_labels, weights=None, conf_type='avg', iou_thr=0.9, skip_box_thr=0.15)
            all_fr_boxes.append(tta_fr_boxes)    
            all_fr_scores.append(tta_fr_scores)
            all_fr_labels.append(tta_fr_labels) # Wheat --> 1
            
        for d2 in detectrons2:         
            tta_d2_boxes, tta_d2_scores, tta_d2_labels = list(), list(), list()
            for i in range(4):
                img_ = TTAImage(img, i) # (1024, 1024, 3), numpy, 0-255
                        
                # Current Detectron2 prediction using TTA
                d2_prediction = d2(img_) # batch size = 1
                d2_boxes = d2_prediction['instances'].get_fields()['pred_boxes'].tensor.data.cpu().numpy()
                d2_scores = d2_prediction['instances'].get_fields()['scores'].data.cpu().numpy().tolist()

                for _ in range(3-i): # Reordering the boxes
                    d2_boxes = rotBoxes90(d2_boxes, img_.shape[1], img_.shape[1])
                
                tta_d2_boxes.append((d2_boxes / img_.shape[1]).tolist())
                tta_d2_scores.append(d2_scores)
                tta_d2_labels.append([1] * len(d2_boxes))
                        
            tta_d2_boxes, tta_d2_scores, tta_d2_labels = weighted_boxes_fusion(tta_d2_boxes, tta_d2_scores, tta_d2_labels, weights=None, conf_type='avg', iou_thr=0.9, skip_box_thr=0.15)
            all_d2_boxes.append(tta_d2_boxes)    
            all_d2_scores.append(tta_d2_scores)
            all_d2_labels.append(tta_d2_labels) # Wheat --> 1
        
        for i in range(4):
            img_ = TTAImage(img, i) # (1024, 1024, 3), numpy, 0-255
            
            # Current Detectron2 prediction using TTA
            y_boxes, y_scores = detect1Image_aug(img_, img_.shape[1], yolov5, device, score_thresh[2], iou_thresh[2])
            y_scores = [s.cpu().item() for s in y_scores]

            for _ in range(3-i): # Reordering the boxes
                y_boxes = rotBoxes90(y_boxes, img_.shape[1], img_.shape[1])

            y_boxes = [b / img_.shape[1] for b in y_boxes] # Normalizing the boxes
            all_y_boxes.append(y_boxes)    
            all_y_scores.append(y_scores)
            all_y_labels.append([1] * len(y_boxes)) # Wheat --> 1
                            
        # Gets predictions of the main three models (5-fold Faster RCNN and Detectron2 models, and a YOLOv5 predictions, using TTA augmentations)
        # Gets current batch average predictions by the faster_rcnn models
        fr_boxes, fr_scores, fr_labels = weighted_boxes_fusion(all_fr_boxes, all_fr_scores, all_fr_labels, weights=fr_f_measures, conf_type='avg', iou_thr=iou_thresh[0], skip_box_thr=score_thresh[0])

        # Gets current batch average predictions by the detectron2 predictors
        d2_boxes, d2_scores, d2_labels = weighted_boxes_fusion(all_d2_boxes, all_d2_scores, all_d2_labels, weights=d2_f_measures, conf_type='avg', iou_thr=iou_thresh[1], skip_box_thr=score_thresh[1])

        # Gets current batch average predictions by the YOLOv5 single model
        y_boxes, y_scores, y_labels = weighted_boxes_fusion([y_boxes], [y_scores], [[1] * len(y_boxes)], weights=None, conf_type='avg', iou_thr=0.9, skip_box_thr=0.15)
         
        # Mixing all of the models
        boxes = [y_boxes, fr_boxes, d2_boxes]
        scores = [y_scores, fr_scores, d2_scores]
        labels = [y_labels, fr_labels, d2_labels]

        # Gets the prediction of the ensemble, containing Faster RCNN, Detectron2 and YOLOv5 models
        boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=[1, 0, 0], conf_type='max', iou_thr=iou_thresh[3], skip_box_thr=score_thresh[3])

        resized_boxes = np.array([])
        if len(boxes) > 0:
            resized_boxes = np.asarray(np.asarray(boxes, dtype=np.float64) * img.shape[1], dtype=np.int32).clip(min=0, max=1023) # Resizing boxes to original size
            scores_str = ['{0:.2f}'.format(s) for s in scores]
            img = draw_bboxes(img, resized_boxes, scores_str, 1.5, thickness=5) # Draws bboxes within the current image
                
            # Converting from Pascal_voc to Coco format
            resized_boxes[:, 2] = resized_boxes[:, 2] - resized_boxes[:, 0]
            resized_boxes[:, 3] = resized_boxes[:, 3] - resized_boxes[:, 1]

        result = {
            'image_id': image_ids[0],
            'PredictionString': format_prediction_string(resized_boxes, scores),
            'image': img
        }
        results.append(result)

    return results

In [ ]:
results = test_ensemble(rcnns, detectrons, yolov5, wheat_testloader, F_MEASURES_PATH_FR, F_MEASURES_PATH_D2)

In [ ]:
plt.imshow(results[3]['image'])

In [ ]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df

In [ ]:
test_df.to_csv('submission.csv', index=False)